# Importing Initial Libraries

In [1]:
import numpy as np
import pandas as pd

# Data Preprocessing

## Read Data

In [2]:
df = pd.read_csv('Data/Input/train.csv')
df.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


## Break Into Training and Target Set

In [3]:
# Break into training/target set
target_column_name = 'median_house_value'
traincols = []
for column in df.columns:
    if (column != target_column_name):
        traincols.append(column)


x_train = df[traincols]
y_train = df[target_column_name]
x_train.describe()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100


## One Hot Encode Categorical Columns

In [4]:
# Union Function
def union(lst1, lst2):
    final_list = list(set(lst1) | set(lst2))
    return final_list

# Check DType and Get Categorical Columns
categorical_columns = []
for column in df.columns:
    if (df[column].dtype != np.float64 and df[column].dtype != np.int64 and df[column].dtype != np.int32 and df[column].dtype != np.float32):
        categorical_columns.append(column)

# One Hot Encode (Remove Last)
encoded_columns = []
for column in categorical_columns:
    prefix = 'one_hot_encoded_' + column + '_'
    x_train = pd.get_dummies(x_train, columns=[column] , prefix=prefix)
    encoded_columns = union(encoded_columns, [i for i in list(x_train.columns) if prefix in i][:-1])
    
not_encoded_columns = [i for i in list(x_train.columns) if "one_hot_encoded_" not in i]
x_train = x_train[union(encoded_columns, not_encoded_columns)]

# Fix for XGBoost
for column in list(x_train.columns):
    if ("<" in column):
        x_train.rename(index=str, columns={column: column.replace("<", "")}, inplace=True)

x_train.describe()

,population,total_bedrooms,households,one_hot_encoded_ocean_proximity__NEAR BAY,one_hot_encoded_ocean_proximity__ISLAND,one_hot_encoded_ocean_proximity__INLAND,median_income,total_rooms,longitude,latitude,one_hot_encoded_ocean_proximity__1H OCEAN,housing_median_age
count,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,1425.476744,537.870553,499.539680,0.110950,0.000242,0.317393,3.870671,2635.763081,-119.569704,35.631861,0.442636,28.639486
std,1132.462122,421.385070,382.329753,0.314077,0.015563,0.465473,1.899822,2181.615252,2.003532,2.135952,0.496710,12.585558
min,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.499900,2.000000,-124.350000,32.540000,0.000000,1.000000
25%,787.000000,296.000000,280.000000,0.000000,0.000000,0.000000,2.563400,1447.750000,-121.800000,33.930000,0.000000,18.000000
50%,1166.000000,435.000000,409.000000,0.000000,0.000000,0.000000,3.534800,2127.000000,-118.490000,34.260000,0.000000,29.000000
75%,1725.000000,647.000000,605.000000,0.000000,0.000000,1.000000,4.743250,3148.000000,-118.010000,37.710000,1.000000,37.000000
max,35682.000000,6445.000000,6082.000000,1.000000,1.000000,1.000000,15.000100,39320.000000,-114.310000,41.950000,1.000000,52.000000


## Impute Null Values

In [10]:
from sklearn.impute import SimpleImputer as Imputer

imputer = Imputer(strategy='mean', copy=False)
x_train = pd.DataFrame(data=imputer.fit_transform(x_train), columns=list(x_train.columns))

x_train.describe()

,population,total_bedrooms,households,one_hot_encoded_ocean_proximity__NEAR BAY,one_hot_encoded_ocean_proximity__ISLAND,one_hot_encoded_ocean_proximity__INLAND,median_income,total_rooms,longitude,latitude,one_hot_encoded_ocean_proximity__1H OCEAN,housing_median_age
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,1425.476744,537.870553,499.539680,0.110950,0.000242,0.317393,3.870671,2635.763081,-119.569704,35.631861,0.442636,28.639486
std,1132.462122,419.266592,382.329753,0.314077,0.015563,0.465473,1.899822,2181.615252,2.003532,2.135952,0.496710,12.585558
min,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.499900,2.000000,-124.350000,32.540000,0.000000,1.000000
25%,787.000000,297.000000,280.000000,0.000000,0.000000,0.000000,2.563400,1447.750000,-121.800000,33.930000,0.000000,18.000000
50%,1166.000000,438.000000,409.000000,0.000000,0.000000,0.000000,3.534800,2127.000000,-118.490000,34.260000,0.000000,29.000000
75%,1725.000000,643.250000,605.000000,0.000000,0.000000,1.000000,4.743250,3148.000000,-118.010000,37.710000,1.000000,37.000000
max,35682.000000,6445.000000,6082.000000,1.000000,1.000000,1.000000,15.000100,39320.000000,-114.310000,41.950000,1.000000,52.000000


### Feature Scaling

In [11]:
from sklearn.preprocessing import RobustScaler as scaler

sc_X = scaler(copy=False)
x_train[not_encoded_columns] = sc_X.fit_transform(x_train[not_encoded_columns])

x_train.describe()

,population,total_bedrooms,households,one_hot_encoded_ocean_proximity__NEAR BAY,one_hot_encoded_ocean_proximity__ISLAND,one_hot_encoded_ocean_proximity__INLAND,median_income,total_rooms,longitude,latitude,one_hot_encoded_ocean_proximity__1H OCEAN,housing_median_age
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,2.064000e+04,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,0.276628,0.288435,0.278584,0.110950,0.000242,0.317393,1.540799e-01,0.299228,-0.284882,0.362926,0.442636,-0.018974
std,1.207316,1.210878,1.176399,0.314077,0.015563,0.465473,8.715378e-01,1.283114,0.528636,0.565067,0.496710,0.662398
min,-1.239872,-1.262094,-1.255385,0.000000,0.000000,0.000000,-1.392252e+00,-1.249816,-1.546174,-0.455026,0.000000,-1.473684
25%,-0.404051,-0.407220,-0.396923,0.000000,0.000000,0.000000,-4.456270e-01,-0.399500,-0.873351,-0.087302,0.000000,-0.578947
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.018608e-16,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.595949,0.592780,0.603077,0.000000,0.000000,1.000000,5.543730e-01,0.600500,0.126649,0.912698,1.000000,0.421053
max,36.797441,17.348736,17.455385,1.000000,1.000000,1.000000,5.259674e+00,21.875018,1.102902,2.034392,1.000000,1.210526


## Train Test Split

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=1)
x_train.describe()

,population,total_bedrooms,households,one_hot_encoded_ocean_proximity__NEAR BAY,one_hot_encoded_ocean_proximity__ISLAND,one_hot_encoded_ocean_proximity__INLAND,median_income,total_rooms,longitude,latitude,one_hot_encoded_ocean_proximity__1H OCEAN,housing_median_age
count,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000
mean,0.276394,0.291806,0.281826,0.111252,0.000303,0.314499,0.156593,0.301480,-0.286092,0.363014,0.444586,-0.020817
std,1.198035,1.209562,1.176737,0.314454,0.017399,0.464330,0.867759,1.280060,0.529704,0.565367,0.496935,0.662423
min,-1.239872,-1.262094,-1.255385,0.000000,0.000000,0.000000,-1.392252,-1.249816,-1.532982,-0.455026,0.000000,-1.473684
25%,-0.405117,-0.407220,-0.400000,0.000000,0.000000,0.000000,-0.441659,-0.397589,-0.875989,-0.087302,0.000000,-0.578947
50%,-0.002132,0.000000,0.003077,0.000000,0.000000,0.000000,0.004473,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.593817,0.595668,0.603846,0.000000,0.000000,1.000000,0.557470,0.601088,0.126649,0.912698,1.000000,0.421053
max,36.797441,16.670036,15.227692,1.000000,1.000000,1.000000,5.259674,21.875018,1.102902,2.034392,1.000000,1.210526


# Exploratory Data Analysis

## Import Libraries

In [13]:
# Import Plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

# Import Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Import Evaluation Metrics
from sklearn.model_selection import cross_val_score

In [14]:
clf = RandomForestRegressor()
clf.fit(x_train, y_train)
y_train_pred = clf.predict(x_train)

trace0 = go.Scatter(
    x = [i for i in range(x_train.shape[0])][:150],
    y = y_train[:150],
    mode = 'lines',
    name = 'y_train'
)
trace1 = go.Scatter(
    x = [i for i in range(x_train.shape[0])][:150],
    y = y_train_pred[:150],
    mode = 'lines',
    name = 'y_train_pred'
)

# y_test_pred = clf.predict(x_test)
# trace2 = go.Scatter(
#     x = [i for i in range(x_train.shape[0])],
#     y = y_test_pred,
#     mode = 'lines',
#     name = 'y_test_pred'
# )

data = [trace0, trace1]
py.iplot(data, filename='line-mode')





c:\anaconda3\envs\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



## Show Variable Corelation

In [15]:
# Visualise Data
from Visualisation.Visualisation import Visualisation 

# Show Variable Corelation
vs = Visualisation(x_train, y_train, target_column_name, [])
vs.show_variable_corelation()

<Figure size 1200x1000 with 2 Axes>

## Show Feature Importance

In [16]:
# Random Forest Feature Importance
clf = GradientBoostingRegressor(n_estimators=100)
clf.fit(x_train, y_train)
fi = clf.feature_importances_
data = [go.Bar(
        x=list(x_train.columns),
        y=list(fi)
    )]

layout = go.Layout(
    title='Random Forest Feature Importance',
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Random Forest Feature Importance.html')

# Gradient Boosting Feature Importance
clf = GradientBoostingRegressor(n_estimators=100)
clf.fit(x_train, y_train)
fi = clf.feature_importances_
data = [go.Bar(
        x=list(x_train.columns),
        y=list(fi)
    )]

layout = go.Layout(
    title='Gradient Boosting Feature Importance',
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Gradient Boosting Feature Importance.html')




# Model Evaluation

## Import Libraries

In [17]:
# Import Plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

# Import Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Import Evaluation Metrics
from sklearn.model_selection import cross_val_score


## Show Cross Validated Scores

In [18]:
cv = 10

clf = LinearRegression()
cv_train_score = cross_val_score(estimator=clf, X=x_train, y=y_train, cv=cv, n_jobs=-1)
cv_test_score = cross_val_score(estimator=clf, X=x_test, y=y_test, cv=cv, n_jobs=-1)
print('Linear Regression Cross Validated Score => Train: ' + str(np.mean(cv_train_score)) + " Test: " + str(np.mean(cv_test_score)))

clf = RandomForestRegressor()
cv_train_score = cross_val_score(estimator=clf, X=x_train, y=y_train, cv=cv, n_jobs=-1)
cv_test_score = cross_val_score(estimator=clf, X=x_test, y=y_test, cv=cv, n_jobs=-1)
print('Random Forest Cross Validated Score => Train: ' + str(np.mean(cv_train_score)) + " Test: " + str(np.mean(cv_test_score)))

clf = KNeighborsRegressor()
cv_train_score = cross_val_score(estimator=clf, X=x_train, y=y_train, cv=cv, n_jobs=-1)
cv_test_score = cross_val_score(estimator=clf, X=x_test, y=y_test, cv=cv, n_jobs=-1)
print('KNN Cross Validated Score => Train: ' + str(np.mean(cv_train_score)) + " Test: " + str(np.mean(cv_test_score)))

clf = GradientBoostingRegressor()
cv_train_score = cross_val_score(estimator=clf, X=x_train, y=y_train, cv=cv, n_jobs=-1)
cv_test_score = cross_val_score(estimator=clf, X=x_test, y=y_test, cv=cv, n_jobs=-1)
print('Gradient Boosting Cross Validated Score => Train: ' + str(np.mean(cv_train_score)) + " Test: " + str(np.mean(cv_test_score)))



Linear Regression Cross Validated Score => Train: 0.6452277373197931 Test: 0.6327545765234855
Random Forest Cross Validated Score => Train: 0.802747545783453 Test: 0.748609499811548
KNN Cross Validated Score => Train: 0.7088353302363746 Test: 0.6524297179765911
Gradient Boosting Cross Validated Score => Train: 0.7724697322451883 Test: 0.7541699384068001
